# 结论：
1、dftest的user_id中，只有2个没有在dfoff中出现过【2495873，1286474】    
2、dftest有约10万个UM_id，但是其中9万个在nf2中没有记录  
3、同上，valid有约21万个UM_id,但是其中18.5万个id在nf2中没有记录  

In [1]:
# 目前只有两个特征比较有效，希望从这两个特征入手，看看核销的人有什么特征

In [2]:
import sys
sys.path.append('../app/')
import fileio
import featureprocess
import preprocess
import matplotlib.pyplot as plt

import preprocess
import featureprocess
import fileio
import evaluation
import numpy as np
import modelacc
import pandas as pd
from sklearn.impute import SimpleImputer

In [3]:
dfoff,dftest,dfon = fileio.read_all()
dfoff,dfall,train,valid = preprocess.preprocess(dfoff)

# nf1和nf2 

In [4]:
def merge_nf2(df):
    """
    输入dftest或者dfoff,用于合并新特征1.
    新特征是使用dfoff的用户id计算的，所以对于其余df可能无法完全兼容，比如dftest会多两个id没有历史数据
    这里会用most填充
    核销数：{0:'U-M没有历史关系';
             1：'有过label-1或者label0
             2:''
            '}
    """
    
    df.loc[:,'UM_id'] = df.User_id.astype('str')+'_'+df.Merchant_id.astype('str')
    dfnf2 = fileio.read_nf2()
    dfobj = pd.merge(df,dfnf2,'left',left_on='UM_id',right_on='用户id')
    
    #imp_most = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    #dfobj.iloc[:,-5::] = imp_most.fit_transform(dfobj.iloc[:,-5::])
    
    return dfobj

def merge_nf1(df):
    """
    输入dftest或者dfoff,用于合并新特征1.
    新特征是使用dfoff的用户id计算的，所以对于其余df可能无法完全兼容，比如dftest会多两个id没有历史数据
    这里会用most填充
    """
    dfnf1 = fileio.read_nf1()
    dfobj = pd.merge(df,dfnf1,'left',left_on='User_id',right_on='用户id')
    
    #imp_most = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    #dfobj.iloc[:,-6::] = imp_most.fit_transform(dfobj.iloc[:,-6::])
    
    return dfobj

In [5]:
dfoff_1  = merge_nf1(dfoff)
dfall_1  = merge_nf1(dfall)

train_1  = merge_nf1(train)
valid_1  = merge_nf1(valid)
dftest_1 = merge_nf1(dftest)

In [6]:
dfoff_2  = merge_nf2(dfoff)
dfall_2  = merge_nf2(dfall)

train_2  = merge_nf2(train)
valid_2  = merge_nf2(valid)
dftest_2 = merge_nf2(dftest)

In [7]:
def count_nanum(df):
    nna = len(df.核销数[df.核销数>=1])
    nle = len(df)
    
    print(nle,nna,nna/nle*100)

In [11]:
valid_2.label.unique()

array([0, 1], dtype=int64)

In [8]:
df0 = valid_1[valid_1.label == 0]
df1 = valid_1[valid_1.label == 1]

count_nanum(df0)
count_nanum(df1)

229715 18636 8.112661341227174
22871 6137 26.8331074286214


In [9]:
df0 = valid_2[valid_2.label == 0]
df1 = valid_2[valid_2.label == 1]
    
count_nanum(df0)
count_nanum(df1)

229715 7239 3.151296171342751
22871 4878 21.328319706178128


In [ ]:
end

In [13]:
dfoff = featureprocess.process_features_main(dfoff)
dfall = featureprocess.process_features_main(dfall)

train = featureprocess.process_features_main(train)
valid = featureprocess.process_features_main(valid)
dftest = featureprocess.process_features_main(dftest)

In [17]:
# 各个特征，柱状图
valid.weekday

0         6
1         1
2         1
3         1
4         4
         ..
252581    1
252582    2
252583    1
252584    1
252585    1
Name: weekday, Length: 252586, dtype: int64